<h1>Imports and Initial Variable Declarations</h1>

In [161]:
# BeautifulSoup and requests library import
from bs4 import BeautifulSoup
import requests 
import re

In [162]:
#page url
page_link = "https://medium.com/@veronicawalsing/how-to-love-something-for-13-years-35cdee86a383"

In [163]:
#page request
page_response = requests.get(page_link, timeout=5)

In [164]:
#parsing content from page and storing it
page_content = BeautifulSoup(page_response.content, "html.parser")

In [165]:
#empty list to store user data
data = []

<h1>Finding and Adding Title to Data</h1>

In [166]:
#grabs div that stores title info
titleInfo = page_content.find_all('div', attrs={"class":"n p"})

In [167]:
#finds title within titleInfo div
href = str(titleInfo).index("<h1")
start = str(titleInfo)[href:].index(">")+href+1
end = str(titleInfo)[href:].index("</h1>")+href

#appends title to data list after removing extra tags
title = re.sub(re.compile('<.*?>'), '', str(titleInfo)[start:end])
data.append(title)

<h1>Finding and Adding Author to Data</h1>

In [168]:
#grabs div that stores user info
userInfo = page_content.find_all('div',attrs={"class":"o n"})

In [169]:
#finds authors name within userInfo div
href = str(userInfo)[24:].index("<a class=")+24
start = str(userInfo)[href:].index(">")+href+1
end = str(userInfo)[href:].index("</a>")+href

#appends authors name to data list
author = str(userInfo)[start:end]
data.append(author)

<h1>Finding and Adding Author Page URL to Data</h1>

In [170]:
#finds date within userInfo div
href = str(userInfo)[24:].index("<a class=")
start = str(userInfo)[href:].index("href=")+href+6
end = str(userInfo)[start:].index("?source")+start

#appends date to data list
url = "medium.com" + str(userInfo)[start:end]
data.append(url)

<h1>Pulling User Profile Page</h1>

In [171]:
#user url
user_link = "https://" + data[2]

In [172]:
#user page request
user_response = requests.get(user_link, timeout=5)

In [173]:
#parsing content from user page to store follower count
user_content = BeautifulSoup(user_response.content, "html.parser")

In [174]:
#user tag
user_href = re.sub("medium.com", "", data[2])

<h1>Finds and adds member since data to data list</h1>

In [175]:
memberInfo = user_content.find_all("span")
year = re.findall("[0-9]{4}", str(memberInfo))[0]
data.append(year)

<h1>Finds and adds following count to data list</h1>

In [176]:
#returns following count and adds to data list
followingInfo = user_content.find_all('a', attrs={"href":user_href+"/following"})
following = re.sub("[^0-9]", "", str(followingInfo))
data.append(following)

<h1>Finds and adds follower count to data list</h1>

In [177]:
#returns follower count and adds to data list
followerInfo = user_content.find_all('a', attrs={"href":user_href+"/followers"})
followers = re.sub("[^0-9]", "", str(followerInfo))
data.append(followers)

<h1>Finding and Adding Date to Data</h1>

In [178]:
#finds date within userInfo div
href = str(userInfo)[end:].index("<a class=")+end
start = str(userInfo)[href:].index(">")+href+1
end = str(userInfo)[href:].index("</a>")+href

#appends date to data list
date = str(userInfo)[start:end]
data.append(date)

<h1>Finding and Adding Number of Claps to Data</h1>

In [179]:
#grabs all paragraphs in the article
clapInfo = page_content.find_all('div',attrs={"class":"n o"})

In [180]:
#finds claps within clapInfo div
result = re.sub(re.compile('<.*?>'), '', str(clapInfo))
result = re.findall("\d+", result)

#appends claps to data list
if len(result) == 0:
    data.append('0')

else:
    data.append(result[0])

<h1>Adding a List of the Text Content to Data</h1>

In [181]:
#grabs all paragraphs in the article
textContent = page_content.find_all("p")
textInfo = str(textContent)

#RegEx that cleans paragraph data by removing html tags and extra commas
result = re.sub(re.compile('<.*?>'), '', textInfo)
result = re.split("\., ", result)
result = result[:-1]
data.append(result)

<h1>Finding and adding a list of tags to data</h1>

In [182]:
tagInfo = page_content.find_all("ul")
tags = re.sub(re.compile('<.*?>'), '', str(tagInfo))
tags = re.sub("]", "", tags)
tags = re.findall('[A-Z][^A-Z]*', tags)
data.append(tags)
data

['How to Love Something for 13 Years',
 'Veronica Walsingham',
 'medium.com/@veronicawalsing',
 '2019',
 '5',
 '273',
 'Nov 11',
 '10',
 ['[When I was younger, I was that annoying friend who asked questions like, “What’s your favorite part about life?” I still remember my friend Angie’s answer. She said, very simply, “Firsts.”, Angie explained that the first time you do something is always the best time because it’s the way you’ll never experience it again. Every other time you do that thing, you’ll do it with the knowledge of what it’s like to do it, but the first time, it’s pure experience',
  'It was such a good answer that I’ve remembered it all this time. But here’s something I don’t remember: the first time I watched Sex and the City. As I played the first episode, there was no visceral memory of the first time I had watched it. In a way, I like this because it’s like I never watched Sex and the City for a first time, but rather it’s a show in my DNA',
  'But I also don’t like th

The data list is in the order <strong>[Title, Author, Author Page, Member Since, Following Count, Follower Count, Data Published, Number of Claps, Text Content, List of Tags] </strong>